In [1]:
from problem import ProblemSingleElementWise
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from callback import UpdatePopulationCallback
import numpy as np
from tools import Tools
from survival import MySurvival
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import warnings
import random
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
import pandas as pd
import logging
from terminator import MyTermination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination.default import DefaultSingleObjectiveTermination, DefaultMultiObjectiveTermination
import testSetup
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.crossover.sbx import SBX
from mutation import IntegerPolynomialMutation
from testSetup import Setup
from pymoo.core.duplicate import ElementwiseDuplicateElimination

warnings.filterwarnings("ignore", ".*feasible.*")
logging.getLogger('matplotlib').setLevel(logging.WARNING)

DEBUG:matplotlib:CACHEDIR=C:\Users\matte\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\matte\.matplotlib\fontlist-v390.json


In [2]:
from problem import ProblemMultiNoConstElementWise, ProblemMultiElementWise

# setup population size and number of events per trace
pop_size = 3000
trace_length = 50



# setup shared components and initial population
(
    encoder, declare, event_log, dataframe, activities_name
) = Setup.initialize_shared_components(path_to_declareModel="../declare_models/model1.decl", trace_length=trace_length)
(
    initial_population, initial_encoded_pop, lower_bounds, upper_bounds, mutation, crossover, sampling
) = Setup.setup_initial_population(activities_name=activities_name, encoder=encoder)


class MyDuplicateElimination(ElementwiseDuplicateElimination):

    def is_equal(self, a, b):
        return a.X[0] == b.X[0]

# setup termination
termination_multi = DefaultMultiObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=0.0025,
    period=30,
    n_max_gen=1000,
    n_max_evals=100000
)
termination_single = DefaultSingleObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=1e-6,
    period=20,
    n_max_gen=1000,
    n_max_evals=100000
)
termination = MyTermination(n_required=pop_size*0.5)


# setup problem
multi_problem = ProblemMultiElementWise(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)
single_problem = ProblemSingleElementWise(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

# setup algorithm
multi_algorithm = NSGA2(
    problem=multi_problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=UpdatePopulationCallback(),
    eliminate_duplicates=True,
)

single_algorithm = GA(
    problem=single_problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=UpdatePopulationCallback(),
    eliminate_duplicates=True,
)

Tools.calculate_overall_diversity(encoder,
                                  "results/encoded_traces/encoded_traces_2025-02-15/ID_3_run_1_multi_yes_constraints.csv", 50)

print(len(declare.activities))

Overall Diversity Score: 0.8186018977777778
13


In [3]:
result = minimize(single_problem, single_algorithm, termination=termination, seed=1, verbose=True)

Tools.save_simple_solution(result.pop, encoder, result.exec_time, "single", pop_size, trace_length)

data = result.algorithm.callback.get_data()

diversity_scores = data.get("diversity_history", None)
constraint_scores = data.get("constraint_history", None)
n_violations_scores = data.get("n_violations_history", None)
n_generations = data.get("generations", None)

print(f"Diversity score:{diversity_scores[-1]}")

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |  0.000000E+00 |  0.000000E+00 | -4.074000E+01 | -4.210000E+01
     2 |     3010 |  0.000000E+00 |  4.6686666667 | -4.082941E+01 | -4.210000E+01
     3 |     6010 |  0.000000E+00 |  4.2133333333 | -4.082941E+01 | -4.210000E+01
     4 |     9010 |  0.000000E+00 |  3.9520000000 | -4.082941E+01 | -4.210000E+01
     5 |    12010 |  0.000000E+00 |  3.6793333333 | -4.101161E+01 | -4.359067E+01
     6 |    15010 |  0.000000E+00 |  3.5800000000 | -4.119018E+01 | -4.458300E+01
     7 |    18010 |  0.000000E+00 |  3.5196666667 | -4.119018E+01 | -4.458300E+01
     8 |    21010 |  0.000000E+00 |  3.4486666667 | -4.124206E+01 | -4.458300E+01
     9 |    24010 |  0.000000E+00 |  3.3626666667 | -4.125309E+01 | -4.458300E+01
    10 |    27010 |  0.000000E+00 |  3.2776666667 | -4.126728E+01 | -4.458300E+01
    11 |    30010 |  0.000000E+00 |  3.2050000000 | -4.126728E+01 | -4.458300E+01
    12 |    3301